In [1]:
# Imports
import torch
import torch.nn as nn 
from torchvision import transforms
import numpy as np
from  matplotlib import pyplot as plt
import torch.nn.functional as F
import time
from utils import get_data, show_batch

In [2]:
# Class for storing things such as learning rate, image size...
class Args:
    def __init__(self):
        self.lr = 2e-4
        self.epochs = 200
        self.b1 = 0.5
        self.b2 = 0.999
        self.latent_dim = 100
        self.img_size = 64
        self.pixels = int(self.img_size ** 2)
        self.channels = 3
        self.img_tuple = (self.channels, self.img_size, self.img_size)
        self.batch_size = 32
        self.g_fmap_size = 64 
        self.d_fmap_size = 64
        self.d_loss_threshold = 0.05

In [3]:
# Loading the images.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
args = Args()
transform = transforms.Compose([transforms.Resize(args.img_size),
                                transforms.CenterCrop(args.img_size),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
paintings, photos, painting_loader, photo_loader = get_data(transform=transform, batch_size=args.batch_size)

In [ ]:
# Defining the CycleGAN. 
class ResidualBlock(nn.Module):
    